#### В приложение по доставке продуктов внедрили умную систему рекомендации товаров – предполагается, что такая система поможет пользователям эффективнее работать с приложением и лучше находить необходимые товары.

#### Чтобы проверить эффективность системы рекомендаций, был проведен АБ-тест. В группе 1 оказались пользователи с новой системой рекомендаций, в группе 0 пользователи со старой версией приложения, где нет рекомендации товаров.

#### Задача – оценить, смогла ли новая система рекомендаций принести пользу бизнесу и пользователям приложения и дать аналитическое заключение с ответом на вопрос, стоит ли включать новую систему рекомендаций на всех пользователей.

In [1]:
import pandas as pd
import scipy as scipy
import requests
import scipy.stats as stats
from  scipy.stats import chi2_contingency
from urllib.parse import urlencode

In [2]:
# исходные данные ab_users_data.csv

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/4qY27tJ23G_B1g'
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

ab_users_data = pd.read_csv(download_url) #исходный датафрэйм

In [3]:
# исходные данные ab_orders.csv

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/prdO8FDQx7lzCg'
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

ab_orders = pd.read_csv(download_url) #исходный датафрэйм

In [4]:
# исходные данные ab_products.csv

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/atwrlmB_wu-fHA'
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

ab_products = pd.read_csv(download_url) #исходный датафрэйм

In [5]:
ab_users_data.head()

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1


In [6]:
ab_users_data.action.unique()

array(['create_order', 'cancel_order'], dtype=object)

In [7]:
ab_products.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


In [8]:
ab_orders.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21.000000,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27.000000,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56.000000,"{74, 6}"
4,1259,2022-08-26 00:03:37.000000,"{20, 45, 67, 26}"


**Предварительная подготовка данных**

Проверим исходные данные на наличие пропущенных значений и типы данных

In [9]:
ab_orders.isna().sum()

order_id         0
creation_time    0
product_ids      0
dtype: int64

In [10]:
ab_users_data.isna().sum()

user_id     0
order_id    0
action      0
time        0
date        0
group       0
dtype: int64

In [11]:
ab_products.isna().sum()

product_id    0
name          0
price         0
dtype: int64

In [12]:
ab_orders.dtypes

order_id          int64
creation_time    object
product_ids      object
dtype: object

In [13]:
ab_users_data.dtypes

user_id      int64
order_id     int64
action      object
time        object
date        object
group        int64
dtype: object

In [14]:
ab_products.dtypes

product_id      int64
name           object
price         float64
dtype: object

Приводим столбцы с датами и временем в нужный формат данных

In [15]:
ab_users_data[['time','date']] = ab_users_data[['time','date']].apply(pd.to_datetime) 

In [16]:
ab_orders.creation_time = pd.to_datetime(ab_orders.creation_time)

В таблице ab_orders преобразуем столбец product_ids в список, чтобы в дальнейшем можно было рассчитать сумму по каждому заказу

In [17]:
ab_orders.product_ids = ab_orders.product_ids.apply(lambda x: x.strip('}{')).str.split(',')

In [18]:
ab_orders.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"[75, 22, 53, 84]"
1,1256,2022-08-26 00:02:21,"[56, 76, 39]"
2,1257,2022-08-26 00:02:27,"[76, 34, 41, 38]"
3,1258,2022-08-26 00:02:56,"[74, 6]"
4,1259,2022-08-26 00:03:37,"[20, 45, 67, 26]"


Далее, с помощью функции explode разобъем списки в столбце product_ids по элементам. Заодно переименуем столбец так, чтобы его наименование совпадало с наименованием в таблице ab_products

In [19]:
ab_orders = ab_orders.explode('product_ids').rename(columns = {'product_ids' : 'product_id'})

In [20]:
ab_orders.product_id = ab_orders.product_id.astype(int) # тип данных у столбцов не совпал, преобразуем

In [21]:
ab_orders.dtypes

order_id                  int64
creation_time    datetime64[ns]
product_id                int64
dtype: object

Объединим таблицы ab_orders и ab_products, чтобы иметь возможность рассчитать сумму по каждому заказу

In [22]:
ab_orders_price = ab_orders.merge(ab_products, how = 'inner', on = 'product_id')

In [23]:
ab_orders_price.head()

,order_id,creation_time,product_id,name,price
0,1255,2022-08-26 00:00:19,75,сок ананасовый,120.0
1,1287,2022-08-26 00:31:36,75,сок ананасовый,120.0
2,1403,2022-08-26 03:01:40,75,сок ананасовый,120.0
3,1424,2022-08-26 04:01:22,75,сок ананасовый,120.0
4,1495,2022-08-26 06:04:05,75,сок ананасовый,120.0


Рассчитаем сумму по каждому заказу

In [24]:
orders_sum = ab_orders_price.groupby('order_id', as_index = False) \
                            .agg({'price' : 'sum'}) \
                            .rename(columns = {'price' : 'order_sum'})

In [25]:
orders_sum.head()

,order_id,order_sum
0,1255,408.7
1,1256,250.5
2,1257,310.2
3,1258,85.0
4,1259,228.0


Объединим таблицы ab_users_data и orders_sum. В итоговой таблице у нас будут полные данные о пользователях, их заказах и суммах этих заказов

In [26]:
final_df = ab_users_data.merge(orders_sum, how = 'left', on = 'order_id')

In [27]:
final_df.head()

,user_id,order_id,action,time,date,group,order_sum
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0,408.7
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1,250.5
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0,310.2
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0,85.0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1,228.0


In [28]:
final_df.isna().sum()

user_id      0
order_id     0
action       0
time         0
date         0
group        0
order_sum    0
dtype: int64

**Анализ результатов АВ-теста**

Мы анализируем работу приложения по доставке продуктов и у нас имеются данные о созданных и отмененных заказах и сумма этих заказов. С точки зрения бизнеса успешность сервиса может измеряться в первую очередь суммой среднего чека на пользователя. Чем выше средний чек, тем лучше. Во вторую очередь можно исследовать то, как часто клиенты делали заказы в разных группах. Чем лучше работает система рекомендаций, тем чаще делаются заказы. Можно проанализировать соотношение сделанных заказов и отмененных -  если новый сервис более удобный, то  доля отмен не должна увеличиваться. 

In [29]:
final_df.query('group == "0"').user_id.nunique()  # количество пользователей из группы 0

515

In [30]:
final_df.query('group == "1"').user_id.nunique()  # количество пользователей из группы 1

502

Так как количество пользователей в двух группах примерно одинаковое, проанализируем общую сумму выручки по заказам. Для этого нужно выбрать только тех пользователей, которые заказы сделали и не отменили.

In [31]:
# все пользователи из нулевой группы
final_df_group0 = final_df.query('group == "0"')\
                          .groupby(['user_id', 'order_id', 'action', 'date'], as_index = False)\
                          .agg({'order_sum' : 'sum'}) 

Чтобы удалить из выборки тех пользователей, которые сделали заказ, но потом его отменили, удалим повторяющиеся строки (у таких заказов повторяется order_id)

In [32]:
created_group0 = final_df_group0.drop_duplicates(subset='order_id', keep= False) # пользователи, не отменившие свой заказ

In [33]:
created_group0.head()

,user_id,order_id,action,date,order_sum
0,964,1255,create_order,2022-08-26,408.7
1,964,1257,create_order,2022-08-26,310.2
4,968,1261,create_order,2022-08-26,430.7
5,968,14882,create_order,2022-08-30,870.0
6,968,15704,create_order,2022-08-31,290.0


In [34]:
a = created_group0.order_sum.sum() # сумма всех заказов пользователей из группы 0

Аналогичные действия для другой группы:

In [35]:
# все пользователи из группы 1
final_df_group1 = final_df.query('group == "1"')\
                          .groupby(['user_id','order_id', 'action', 'date'], as_index = False)\
                          .agg({'order_sum' : 'sum'}) 

In [36]:
created_group1 = final_df_group1.drop_duplicates(subset='order_id', keep= False) # пользователи, не отменившие свой заказ

In [37]:
created_group1.head()

,user_id,order_id,action,date,order_sum
0,965,1256,create_order,2022-08-26,250.5
1,965,3946,create_order,2022-08-27,860.6
2,965,9097,create_order,2022-08-29,608.2
3,965,9101,create_order,2022-08-29,203.5
4,965,10401,create_order,2022-08-29,943.7


In [38]:
b = created_group1.order_sum.sum() # сумма всех заказов пользователей из группы 1

In [39]:
print('{:.2f} отношение выручки из группы 1 к выручке из группы 0'.format(b/a))
print('{:.2f} отношение количества заказов в группе 1 к группе 0'.format(created_group1.order_id.nunique()/created_group0.order_id.nunique()))

1.51 отношение выручки из группы 1 к выручке из группы 0
1.56 отношение количества заказов в группе 1 к группе 0


*Вывод 1: пользователи из группы 1 (с новой системой рекомендаций) принесли компании в полтора раза больше выручки.* <br> *Вывод 2: пользователи из группы 1 (с новой системой рекомендаций) заказывали в полтора раза чаще.* 

Проведем анализ среднего чека на пользователя

In [40]:
mean_0 = created_group0.groupby('user_id', as_index = False)\
                          .agg({'order_sum' : 'sum'})

In [41]:
mean_1 = created_group1.groupby('user_id', as_index = False)\
                          .agg({'order_sum' : 'sum'})

In [42]:
mean_0.order_sum.mean()  # средний чек в группе 0 (пользователи, не отменившие заказ)

1139.5623046875

In [43]:
mean_1.order_sum.mean()  # средний чек в группе 1 (пользователи, не отменившие заказ)

1753.7493013972055

In [44]:
1753.7493013972055 / 1139.5623046875

1.5389674563499471

Средний чек в первой группе выше на 53,9%. Проверим, является ли эта разница статистически значимой. Так как данные являются количественными, для анализ будем использовать t-тест.<br>

В рамках t-теста формулируются две гипотезы:<br>
Н_0 – в выборках значимого различия между средними значениями нет<br>
H_1 – средние в выборках не равны (альтернативная гипотеза)<br>
Для сравнения средних с помощью t-критерия нужно, чтобы соблюдался ряд требований, в частности, распределение значений должно быть нормальным, дисперсии внутри групп должны быть примерно одинаковы (требование гомогенности дисперсий).

Проверим две группы данных на соответствие требованиям.

Обычно нормальность тестируют с помощью теста Шапиро-Уилка (scipy.stats.shapiro()), однако на больших выборках этот тест слишком рьяно находит отклонения от нормальности. Поэтому использовалась функция scipy.stats.normaltest() - она больше адаптирована к большим выборкам
Нулевая гипотеза теста - распределение нормальное. Альтернативная гипотеза теста - распределение отличается от нормального

In [45]:
scipy.stats.normaltest(mean_0.order_sum)  # проверка на нормальноссть распределения для нулевой группы. 

NormaltestResult(statistic=165.14145937540576, pvalue=1.3803452652449819e-36)

In [46]:
scipy.stats.normaltest(mean_1.order_sum)  # проверка на нормальноссть распределения для первой группы.

NormaltestResult(statistic=21.24303377815477, pvalue=2.438562133100666e-05)

Полученные значения p-value меньше порога значимости 0,05, следовательно, тесты показывают, что мы не можем принять нулевую гипотезу, значения в группах распределены ненормально.

Тестируем различие в дисперсиях с помощью критерия Левена. Нулевая гипотеза для этого теста: дисперсии в двух выборках не имеют значимых различий. Альтернативная гипотеза - дисперсии в выборках значимо отличаются

In [47]:
stats.levene(mean_0.order_sum, mean_1.order_sum)

LeveneResult(statistic=18.466657739438077, pvalue=1.896012459520485e-05)

Так как p-значение теста Левена меньше принятого уровня значимости 0.05, у нас нет оснований принять нулевую гипотезу и мы делаем вывод о том, что дисперсии в двух выборках имеют значимые различия

В итоге, требования при применении t-критерия не выполяются, распределение не является нормальным и дисперсии отличаются. На практике t-тест может быть использован для сравнения средних и при ненормальном распределении, особенно на больших выборках<br>

По умолчанию stats.ttest_ind предполагает равные дисперсии генеральной совокупности. Если установлен параметр equal_var=False, то это не предполагает равных дисперсий генеральной совокупности (поправка Уэлча).

In [48]:
stats.ttest_ind(mean_0.order_sum, mean_1.order_sum, equal_var=False)

Ttest_indResult(statistic=-11.208629753614105, pvalue=1.572166429481266e-27)

Полученное в результате t-теста p-значение ниже уровня значимости 0.05, следовательно, у нас нет оснований принять нулевую гипотезу, принимаем альтернативную – в группах средние значения отличиются статистически значимо.

Так как у нас есть две категориальные переменные, используем критерий Хи-квадрат <br> Проверим нулевую гипотезу: в двух группах количество созданных и отмененных заказов одинаковое. Альтернативной будет являться гипотеза, что новый алгоритм изменил соотношение созданных и отмененных заказов.

Пользователи, которые создали и не отменили свои заказы есть в таблицах final_df_group0 и final_df_group1. Их количество приведено ниже

In [49]:
create_0 = created_group0.shape[0] # кол-во созданных и не отмененных заказов из группы 0

In [50]:
create_0

1527

In [51]:
create_1 = created_group1.shape[0] # кол-во созданных и не отмененных заказов из группы 1

In [52]:
create_1

2382

Посчитаем пользователей, отменивших свои заказы

In [53]:
canceled_group0 = final_df_group0.loc[final_df_group0.duplicated(['order_id'])]

In [54]:
canceled_group0.head()

,user_id,order_id,action,date,order_sum
3,966,1258,create_order,2022-08-26,85.0
40,993,1296,create_order,2022-08-26,535.8
42,995,1298,create_order,2022-08-26,273.8
55,1008,1315,create_order,2022-08-26,190.0
61,1012,9760,create_order,2022-08-29,685.8


In [55]:
canceled_group1 = final_df_group1.loc[final_df_group1.duplicated(['order_id'])]

In [56]:
canceled_group1.head()

,user_id,order_id,action,date,order_sum
25,973,8736,create_order,2022-08-29,220.8
32,974,37835,create_order,2022-09-04,438.0
42,978,51564,create_order,2022-09-07,133.8
70,988,58492,create_order,2022-09-08,263.0
76,992,13162,create_order,2022-08-30,90.4


In [57]:
cancel_0 = canceled_group0.shape[0] # кол-во отмененных заказов из группы 0

In [58]:
cancel_1 = canceled_group1.shape[0] # кол-во отмененных заказов из группы 1

In [59]:
chi_table = pd.DataFrame({'created':[create_0, create_1], 'canceled':[cancel_0, cancel_1]} , index=["group_0", "group_1"])

In [60]:
chi_table

,created,canceled
group_0,1527,82
group_1,2382,132


In [61]:
chi_table.created.sum()+chi_table.canceled.sum() 
# кол-во наблюдений в таблице chi_table равно кол-ву уникальных order_id в ab_users_data, т. е. 
# ничего не потеряли в процессе

4123

In [62]:
chi2_contingency(chi_table)

(0.021274207290295187,
 0.8840344321879333,
 1,
 array([[1525.48653893,   83.51346107],
        [2383.51346107,  130.48653893]]))

In [63]:
82/1609

0.0509633312616532

In [64]:
132/2514

0.05250596658711217

*Вывод 3: значение p-value = 0.884, что выше порога значимости 0.05, следовательно, оснований для отклонений нулевой гипотезы у нас нет. Новый алгоритм рекомендаций не повлиял существенным образом на количество отмененных заказов* 

**Итоговый вывод:** Внедрение новой системы рекомендаций не повлияло на соотношение количества сделанных и отмененных заказов в группах. При этом, в группе с новой системой рекомендаций средняя сумма чека увеличилась и эта разница является статистически значимой.